In [1]:
from anyio.lowlevel import checkpoint
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

from Pytorch.CNN import learning_rate

In [3]:
dataset= load_dataset('shawhin/phishing-site-classification')

README.md: 0.00B [00:00, ?B/s]

c:\Users\Dawood Khan\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dawood Khan\.cache\huggingface\hub\datasets--shawhin--phishing-site-classification. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


data/train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

In [5]:
checkpoint='google-bert/bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Dawood Khan\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dawood Khan\.cache\huggingface\hub\models--google-bert--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
id2label = {0:'Safe', 1:'Not Safe'}
label2id = {'Safe':0, 'Not Safe':1}
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
for name, param in model.base_model.named_parameters():
    print(name, param.requires_grad)

embeddings.word_embeddings.weight True
embeddings.position_embeddings.weight True
embeddings.token_type_embeddings.weight True
embeddings.LayerNorm.weight True
embeddings.LayerNorm.bias True
encoder.layer.0.attention.self.query.weight True
encoder.layer.0.attention.self.query.bias True
encoder.layer.0.attention.self.key.weight True
encoder.layer.0.attention.self.key.bias True
encoder.layer.0.attention.self.value.weight True
encoder.layer.0.attention.self.value.bias True
encoder.layer.0.attention.output.dense.weight True
encoder.layer.0.attention.output.dense.bias True
encoder.layer.0.attention.output.LayerNorm.weight True
encoder.layer.0.attention.output.LayerNorm.bias True
encoder.layer.0.intermediate.dense.weight True
encoder.layer.0.intermediate.dense.bias True
encoder.layer.0.output.dense.weight True
encoder.layer.0.output.dense.bias True
encoder.layer.0.output.LayerNorm.weight True
encoder.layer.0.output.LayerNorm.bias True
encoder.layer.1.attention.self.query.weight True
encoder.

In [8]:
# Freeze the base model layers if you donot have enough resources
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# Unfreese the base model pooling layers
for name, param in model.base_model.named_parameters():
    if 'pooler' in name:
        param.requires_grad = True

In [9]:
# Now we print the layers to cross check
for name, param in model.base_model.named_parameters():
    print(name, param.requires_grad)

embeddings.word_embeddings.weight False
embeddings.position_embeddings.weight False
embeddings.token_type_embeddings.weight False
embeddings.LayerNorm.weight False
embeddings.LayerNorm.bias False
encoder.layer.0.attention.self.query.weight False
encoder.layer.0.attention.self.query.bias False
encoder.layer.0.attention.self.key.weight False
encoder.layer.0.attention.self.key.bias False
encoder.layer.0.attention.self.value.weight False
encoder.layer.0.attention.self.value.bias False
encoder.layer.0.attention.output.dense.weight False
encoder.layer.0.attention.output.dense.bias False
encoder.layer.0.attention.output.LayerNorm.weight False
encoder.layer.0.attention.output.LayerNorm.bias False
encoder.layer.0.intermediate.dense.weight False
encoder.layer.0.intermediate.dense.bias False
encoder.layer.0.output.dense.weight False
encoder.layer.0.output.dense.bias False
encoder.layer.0.output.LayerNorm.weight False
encoder.layer.0.output.LayerNorm.bias False
encoder.layer.1.attention.self.query

In [10]:
# text_preprocessing function
def preprocessing_function(sentences):
    return tokenizer(sentences['text'], truncation = True)

In [11]:
# Now we tokenize the dataset
tokenized_dataset = dataset.map(preprocessing_function, batched=True)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [12]:
# here we create a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
# Evaluation metrics for the model
accuracy = evaluate.load('accuracy')
auc_score = evaluate.load('roc_auc')

def compute_metrics(eval_pred):
    # Get Predicitons
    predictions, labels = eval_pred
    # applying sotmax to get the probabilities
    probabilities = np.exp(predictions)/np.exp(predictions).sum(-1,keepdims=True)
    # using above probabilities of positive class for AUC_ROC calculation
    positive_class_probabilities = probabilities[:,1]
    # Compute the AUC
    auc = np.round(auc_score.compute(prediction_scores = positive_class_probabilities, references = labels)['roc_auc'],3)
    # predict most probable class
    predicted_class = np.argmax(predictions, axis = 1)
    # compute Accuracy
    accu= np.round(accuracy.compute(predictions = predicted_class, references = labels)['accuracy'],3)
    return {'Accuracy':accu, 'AUC':auc}

# Train the Model

In [21]:
# HyperParmeters
learning_rate = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir = 'Phishing_URL_Detection_BERT',
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    logging_strategy = 'epoch',
    eval_strategy='epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

In [22]:
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = tokenized_dataset['train'],
    eval_dataset= tokenized_dataset['test'],
    tokenizer = tokenizer,
    data_collator = data_collator
)

C:\Users\Dawood Khan\AppData\Local\Temp\ipykernel_16240\273788804.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.323700,0.315200,0.844000,0.939000
2,0.361000,0.316188,0.844000,0.944000
3,0.363500,0.298108,0.867000,0.949000
4,0.329400,0.332561,0.878000,0.950000
5,0.344700,0.293292,0.880000,0.951000
6,0.322800,0.278022,0.871000,0.955000
7,0.319900,0.278009,0.880000,0.953000
8,0.300500,0.277624,0.871000,0.955000
9,0.335700,0.271542,0.878000,0.956000
10,0.312200,0.276292,0.876000,0.956000


TrainOutput(global_step=2630, training_loss=0.3313411495078199, metrics={'train_runtime': 86.5128, 'train_samples_per_second': 242.739, 'train_steps_per_second': 30.4, 'total_flos': 706603239165360.0, 'train_loss': 0.3313411495078199, 'epoch': 10.0})

In [25]:
# Now we will apply the model on the validation data
predictions = trainer.predict(tokenized_dataset['validation'])
# Extracting the logits
logits = predictions.predictions
labels = predictions.label_ids

# Now using the Compute Metrics Function
metrics = compute_metrics((logits,labels))
print(metrics)

{'Accuracy': np.float64(0.882), 'AUC': np.float64(0.948)}


In [26]:
from huggingface_hub import notebook_login
notebook_login()

In [28]:
trainer.push_to_hub()

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/Dawood4893723/Phishing_URL_Detection_BERT/commit/588b5f7c25adba6fe722c0f267347d39d7b8e8c0', commit_message='End of training', commit_description='', oid='588b5f7c25adba6fe722c0f267347d39d7b8e8c0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dawood4893723/Phishing_URL_Detection_BERT', endpoint='https://huggingface.co', repo_type='model', repo_id='Dawood4893723/Phishing_URL_Detection_BERT'), pr_revision=None, pr_num=None)

In [29]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [30]:
model = model.to(device)

In [31]:
input_text = "000mclogin.micloud-object-storage-xc-cos-static-web-hosting-qny.s3.us-east.cloud-object-storage.appdomain.cloud"
inputs = tokenizer(input_text, return_tensors='pt').to(device)

In [35]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim = -1)

# predictions is a tensor of class indices
predicted_ids = predictions.cpu().numpy()  # convert tensor to numpy array if on GPU
predict_labels = [model.config.id2label[i] for i in predicted_ids]

print(f'Predicted label: {predict_labels}')

Predicted label: ['Not Safe']
